In [1]:
import uuid
import pandas as pd

from project_huishoudboekje.config import GeneralSettings as GenSet
from project_huishoudboekje.database import read_sql_table_cats

from project_huishoudboekje.find_category import FindCategory

In [2]:
df_categories = read_sql_table_cats(to_records=False, fill_nan_end_year=True)

In [3]:
df_categories.dtypes

grouplevel    object
category      object
begin_year     int64
end_year       int64
dtype: object

In [4]:
df_categories[df_categories.end_year.isnull()]

,grouplevel,category,begin_year,end_year


In [8]:
df_categories_old = pd.read_excel(GenSet.project_path / 'data/categories.xlsx')

In [9]:
df_categories_old.head()

,CATEGORY,GROUP,2022,2023,2024,2025
0,Boodschappen (food),Aankopen,1,1,1,1
1,Boodschappen (non-food),Aankopen,1,1,1,1
2,Kleding,Aankopen,1,0,0,0
3,Schoenen,Aankopen,1,0,0,0
4,Cadeaus,Aankopen,1,1,1,1


In [6]:
df = pd.read_excel(GenSet.project_path / 'data/processed/transactions_test.xlsx')

In [7]:
df.head()

,2024,2025,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,...,SOURCE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,CATEGORY,ANALYSE_IND,GROUP,2024,2025
0,NaN,NaN,fa393fc6-264a-44bb-ae0f-c2f8d1ab374c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ASN Bank,OVS,credit,Zalando Payments GmbH - 011113919304-DE3410070...,85.49,Kleding en schoenen,1,Aankopen,1.0,1.0
1,NaN,NaN,e035b706-151d-4ef3-a846-0f4df5c918e5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ASN Bank,IDE,debet,Aouled via Tikkie - 001067791992 0031055115556...,75.00,Baby artikelen,1,Aankopen,0.0,1.0
2,NaN,NaN,d12180f1-5998-4a2b-a6ec-2b04475c5803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ASN Bank,IDE,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,50.19,Boodschappen (non-food),1,Aankopen,1.0,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ASN Bank,-,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,8.36,Baby artikelen,1,Aankopen,0.0,1.0
4,NaN,NaN,2f2575a4-0a35-4109-87c1-6780bf772834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ASN Bank,EIC,debet,VOGELBESCHERMING NEDERLAND - Europese incasso:...,5.00,Overige goede doelen,1,Giften,1.0,1.0


In [8]:
df = df[['DATE', 'TRANSACTION_TYPE', 'FINANCIAL_TYPE', 'PARTY', 'AMOUNT', 'ANALYSE_IND']].copy()

In [9]:
# Add unique ID for each transaction
df['TRANS_ID'] = [uuid.uuid4() for _ in range(len(df.index))]

df = df.assign(
    SOURCE='ASN Bank',
    CATEGORY='Niet-gecategoriseerd').set_index('TRANS_ID')

df = FindCategory().assign_category(df)

In [10]:
df.head()

,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,ANALYSE_IND,SOURCE,CATEGORY
TRANS_ID,,,,,,,,
9e856fc8-a2ae-4097-9788-91e0328c0025,2025-07-31,OVS,credit,Zalando Payments GmbH - 011113919304-DE3410070...,85.49,1,ASN Bank,Kleding en schoenen
020496ae-668f-41bb-b093-8482db06dbb9,2025-07-31,IDE,debet,Aouled via Tikkie - 001067791992 0031055115556...,75.00,1,ASN Bank,Baby artikelen
54197e3f-dbfc-45f5-8a6b-9784f1757269,2025-07-31,IDE,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,50.19,1,ASN Bank,Boodschappen (non-food)
eaf7d569-7a3e-465f-b1ed-aa4f10731900,2025-07-31,-,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,8.36,1,ASN Bank,Boodschappen (non-food)
53a7ebd0-5858-4abd-b82f-49dcb2fb0eff,2025-07-31,EIC,debet,VOGELBESCHERMING NEDERLAND - Europese incasso:...,5.00,1,ASN Bank,Overige goede doelen


In [21]:
df_new = df.reset_index().merge(
            df_categories_old, how='left', left_on='CATEGORY', right_on='CATEGORY').rename(
            columns={'index': 'TRANS_ID'}).set_index('TRANS_ID')

In [22]:
df_new.head()

,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,ANALYSE_IND,SOURCE,CATEGORY,GROUP,2022,2023,2024,2025
TRANS_ID,,,,,,,,,,,,,
18fd3156-238b-45b0-adf9-757c63cb52e3,2025-07-31,OVS,credit,Zalando Payments GmbH - 011113919304-DE3410070...,85.49,1,ASN Bank,Kleding en schoenen,Aankopen,0.0,1.0,1.0,1.0
51adb712-4a53-40e2-a8ba-0d03b0e18de8,2025-07-31,IDE,debet,Aouled via Tikkie - 001067791992 0031055115556...,75.00,1,ASN Bank,Baby artikelen,Aankopen,0.0,0.0,0.0,1.0
bb6d548c-03de-438c-b5bc-f5e5267a903f,2025-07-31,IDE,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,50.19,1,ASN Bank,Boodschappen (non-food),Aankopen,1.0,1.0,1.0,1.0
50175f4d-8fba-4728-898b-957bde546b75,2025-07-31,-,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,8.36,1,ASN Bank,Boodschappen (non-food),Aankopen,1.0,1.0,1.0,1.0
1d4cfec5-4750-45be-90ba-d10457b998fb,2025-07-31,EIC,debet,VOGELBESCHERMING NEDERLAND - Europese incasso:...,5.00,1,ASN Bank,Overige goede doelen,Giften,0.0,0.0,1.0,1.0


In [26]:
df_new[df_new.index.duplicated()]

,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,ANALYSE_IND,SOURCE,CATEGORY,GROUP,2022,2023,2024,2025
TRANS_ID,,,,,,,,,,,,,
7b04cc66-2b22-4d92-83c2-8ec863356073,2025-07-30,EIC,debet,AEGON HYPOTHEKEN BV - Europese incasso: Termij...,1261.14,1,ASN Bank,Hypotheek,Woonlasten,0.0,0.0,0.0,1.0
064e0e36-1c59-4a98-b814-95f6a7ff6631,2025-07-28,EIC,debet,Hegg Energie Diensten - Europese incasso: NL-I...,140.02,1,ASN Bank,Energie,Woonlasten,0.0,0.0,0.0,1.0
a057263e-3aaa-4cbb-9c5f-c2f42f67b967,2025-07-28,EIC,debet,Hegg Energie Diensten - Europese incasso: NL-I...,140.02,1,ASN Bank,Energie,Woonlasten,0.0,0.0,0.0,1.0
4e175879-14e8-49a3-a6ac-c0d22018d857,2025-07-25,BEA,debet,HD8201GammaA'foort >AMERSFOOR25.07.2025 16U59,21.70,1,ASN Bank,Onderhoud woning,Woonlasten,0.0,0.0,0.0,1.0
455f4cf3-f74c-4b59-a6f2-d5007ddbc7e5,2025-07-21,EIC,debet,VITENS NV - Europese incasso: NL-Factuurnr,15.00,1,ASN Bank,Water,Woonlasten,0.0,0.0,0.0,1.0
47426c9a-18ca-4881-89ef-d5ad5e939d27,2025-07-18,BEA,debet,Karwei Amersfoort >AMERSFOOR18.07.2025,25.07,1,ASN Bank,Onderhoud woning,Woonlasten,0.0,0.0,0.0,1.0
e41f7400-63ec-44e0-9db9-c95fc0838733,2025-07-16,BEA,debet,Costes Amersfoort >AMERSFOOR16.07.2025,49.95,1,ASN Bank,Onderhoud woning,Woonlasten,0.0,0.0,0.0,1.0
194876b1-9285-41a1-b10e-79e93d608e89,2025-06-27,IDE,debet,OnlineVloerenSpecialist.nl via Stichting Molli...,14.90,1,ASN Bank,Onderhoud woning,Woonlasten,0.0,0.0,0.0,1.0
ed749c1f-4dd6-4e4e-a6e6-0a753c2b5df7,2025-06-27,EIC,debet,AEGON HYPOTHEKEN BV - Europese incasso: Termij...,1261.14,1,ASN Bank,Hypotheek,Woonlasten,0.0,0.0,0.0,1.0


In [11]:
df['YEAR'] = df['DATE'].dt.year

In [43]:
import random

In [47]:
random.randint(0, 3)

1

In [50]:
df['YEAR_NEW'] = df['YEAR'].apply(lambda x: x - random.randint(0, 3)).astype('int32')

In [62]:
df.head()

,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,ANALYSE_IND,SOURCE,CATEGORY,YEAR,YEAR_NEW
TRANS_ID,,,,,,,,,,
18fd3156-238b-45b0-adf9-757c63cb52e3,2025-07-31,OVS,credit,Zalando Payments GmbH - 011113919304-DE3410070...,85.49,1,ASN Bank,Kleding en schoenen,2025,2024
51adb712-4a53-40e2-a8ba-0d03b0e18de8,2025-07-31,IDE,debet,Aouled via Tikkie - 001067791992 0031055115556...,75.00,1,ASN Bank,Baby artikelen,2025,2023
bb6d548c-03de-438c-b5bc-f5e5267a903f,2025-07-31,IDE,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,50.19,1,ASN Bank,Boodschappen (non-food),2025,2025
50175f4d-8fba-4728-898b-957bde546b75,2025-07-31,-,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,8.36,1,ASN Bank,Boodschappen (non-food),2025,2025
1d4cfec5-4750-45be-90ba-d10457b998fb,2025-07-31,EIC,debet,VOGELBESCHERMING NEDERLAND - Europese incasso:...,5.00,1,ASN Bank,Overige goede doelen,2025,2024


In [12]:
df_new_method = df.merge(df_categories, how='left', left_on=['CATEGORY'], right_on=['category'])

In [13]:
df.shape

(954, 9)

In [14]:
df_new_method.shape

(1020, 13)

In [17]:
df_new_method['CHECK_RANGE'] = df_new_method.apply(lambda x: 1 if (((x['YEAR'] >= x['begin_year']) & (x['YEAR'] <= x['end_year'])) | (x['CATEGORY'] == 'Niet-gecategoriseerd')) else 0, axis=1)

In [18]:
df_new_method[df_new_method.CHECK_RANGE == 0]

,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,ANALYSE_IND,SOURCE,CATEGORY,YEAR,grouplevel,category,begin_year,end_year,CHECK_RANGE
6,2025-07-30,EIC,debet,AEGON HYPOTHEKEN BV - Europese incasso: Termij...,1261.14,1,ASN Bank,Hypotheek,2025,Verzorging en gezondheid,Hypotheek,2022.0,2024.0,0
7,2025-07-30,EIC,debet,AEGON HYPOTHEKEN BV - Europese incasso: Termij...,1261.14,1,ASN Bank,Hypotheek,2025,Woning,Hypotheek,2022.0,2024.0,0
10,2025-07-28,EIC,debet,Hegg Energie Diensten - Europese incasso: NL-I...,140.02,1,ASN Bank,Energie,2025,Woning,Energie,2022.0,2024.0,0
13,2025-07-28,EIC,debet,Hegg Energie Diensten - Europese incasso: NL-I...,140.02,1,ASN Bank,Energie,2025,Woning,Energie,2022.0,2024.0,0
33,2025-07-25,BEA,debet,HD8201GammaA'foort >AMERSFOOR25.07.2025 16U59,21.70,1,ASN Bank,Onderhoud woning,2025,Woning,Onderhoud woning,2022.0,2024.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,2025-01-13,BEA,debet,BCK*Praxis >AMERSFOOR13.01.2025 17U13,167.62,1,ASN Bank,Onderhoud woning,2025,Woning,Onderhoud woning,2022.0,2024.0,0
977,2025-01-06,BEA,debet,BCK*Praxis >AMERSFOOR 6.01.2025,18.98,1,ASN Bank,Onderhoud woning,2025,Woning,Onderhoud woning,2022.0,2024.0,0
1006,2025-01-02,EIC,debet,AEGON NEDERLAND - Europese incasso: AEGON,1261.03,1,ASN Bank,Hypotheek,2025,Verzorging en gezondheid,Hypotheek,2022.0,2024.0,0
1007,2025-01-02,EIC,debet,AEGON NEDERLAND - Europese incasso: AEGON,1261.03,1,ASN Bank,Hypotheek,2025,Woning,Hypotheek,2022.0,2024.0,0


In [19]:
df_categories = read_sql_table_cats(to_records=False, fill_nan_end_year=True).rename(
            columns={'grouplevel': 'GROUP', 'category': 'CATEGORY'})

In [20]:
df.shape

(954, 9)

In [22]:
df_test = df.reset_index().merge(
            df_categories, how='left', left_on='CATEGORY', right_on='CATEGORY').rename(
            columns={'index': 'TRANS_ID'})

In [23]:
df_test.shape

(1020, 13)

In [24]:
df_test = df_test[((df_test['CATEGORY'] == 'Niet-gecategoriseerd') | ((df_test['DATE'].dt.year >= df_test['begin_year']) & (
                df_test['DATE'].dt.year <= df_test['end_year'])))].drop(columns=['begin_year', 'end_year']).copy()

In [25]:
df_test.shape

(954, 11)

In [26]:
df_test.head()

,TRANS_ID,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,ANALYSE_IND,SOURCE,CATEGORY,YEAR,GROUP
0,9e856fc8-a2ae-4097-9788-91e0328c0025,2025-07-31,OVS,credit,Zalando Payments GmbH - 011113919304-DE3410070...,85.49,1,ASN Bank,Kleding en schoenen,2025,Aankopen
1,020496ae-668f-41bb-b093-8482db06dbb9,2025-07-31,IDE,debet,Aouled via Tikkie - 001067791992 0031055115556...,75.00,1,ASN Bank,Baby artikelen,2025,Aankopen
2,54197e3f-dbfc-45f5-8a6b-9784f1757269,2025-07-31,IDE,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,50.19,1,ASN Bank,Boodschappen (non-food),2025,Aankopen
3,eaf7d569-7a3e-465f-b1ed-aa4f10731900,2025-07-31,-,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,8.36,1,ASN Bank,Boodschappen (non-food),2025,Aankopen
4,53a7ebd0-5858-4abd-b82f-49dcb2fb0eff,2025-07-31,EIC,debet,VOGELBESCHERMING NEDERLAND - Europese incasso:...,5.00,1,ASN Bank,Overige goede doelen,2025,Giften
